In [ ]:
# test.py
import sqlite3
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler

# Connect to database
DB_PATH = "../data/processed/chess_games.db"
conn = sqlite3.connect(DB_PATH)

# Get first game
query = """
SELECT 
    moves,
    white, black,
    white_elo, black_elo,
    eco, opening,
    result
FROM chess_games 
WHERE moves IS NOT NULL
  AND white_elo IS NOT NULL
  AND black_elo IS NOT NULL
LIMIT 1
"""


row = conn.execute(query).fetchone()

# Print game info
print("Game Information:")
print("-" * 40)
print(f"White: {row[1]} ({row[3]})")
print(f"Black: {row[2]} ({row[4]})")
print(f"ECO: {row[5]}")
print(f"Opening: {row[6]}")
print(f"Result: {row[7]}")
print("\nPGN:")
print(row[0])

# # change row 0 to cut moves till 8th move regex 8.
# moves = row[0].split(" ")[0:22]
# row = list(row)
# row[0] = " ".join(moves)
# row = tuple(row)
# print(row[0])

# Parse game
game = chess.pgn.read_game(io.StringIO(row[0]))

# Initialize feature extractor
feature_extractor = FeatureExtractor()

# Extract features without engine first
# print("\nFeatures without engine analysis:")
# print("-" * 40)
# features = feature_extractor.extract_features(game)
# for name, value in features.__dict__.items():
#     print(f"{name}: {value:.3f}")

# Initialize stockfish and get evaluations
stockfish = StockfishHandler(path="stockfish", depth=20)  # Update path
try:
    # Get positions and engine evaluations
    positions = feature_extractor._get_positions(game)
    print("\nPositions:")
    print(positions)
    print('-'*40)
    evals = [stockfish.evaluate_position(pos, i) for i, pos in enumerate(positions)]
    print("\nEngine evaluations:")
    print(evals)

    # Extract features with engine
    print("\nFeatures with engine analysis:")
    print("-" * 40)
    features_with_engine = feature_extractor.extract_features(game, evals)
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")
finally:
    stockfish.close()

conn.close()

Game Information:
----------------------------------------
White: Abdallah1990 (2510)
Black: EzWin78 (2647)
ECO: D00b
Opening: None
Result: 0-1

PGN:
1.d4 d5 2.c3 Nf6 3.c4 e6 4.Nf3 Bb4+ 5.Bd2 Be7 6.Nc3 O-O 7.cxd5 exd5 8.Bg5 b6 9.e3 Bb7 10.Bd3 Nbd7 11.O-O c5 12.Rc1 c4 13.Bb1 a6 14.Ne5 b5 15.a3 Nxe5 16.dxe5 Nd7 17.Bxe7 Qxe7 18.f4 Rad8 19.Nxd5 Qc5 20.Be4 Bxd5 21.Qxd5 Qxe3+ 22.Kh1 Nf6 23.exf6 Rxd5 24.fxg7 Rfd8 25.Bxd5 Rxd5 26.f5 Kxg7 27.f6+ Kg6 { Normal} 0-1

Positions:
[Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'), Board('rnbqkbnr/pppppppp/8/8/7P/8/PPPPPPP1/RNBQKBNR b KQkq - 0 1'), Board('rnbqkbnr/ppppppp1/8/7p/7P/8/PPPPPPP1/RNBQKBNR w KQkq - 0 2'), Board('rnbqkbnr/ppppppp1/8/7p/6PP/8/PPPPPP2/RNBQKBNR b KQkq - 0 2'), Board('rnbqkbnr/ppppppp1/8/8/6pP/8/PPPPPP2/RNBQKBNR w KQkq - 0 3'), Board('rnbqkbnr/ppppppp1/8/8/6pP/5P2/PPPPP3/RNBQKBNR b KQkq - 0 3'), Board('rnbqkbnr/ppppppp1/8/8/7P/5p2/PPPPP3/RNBQKBNR w KQkq - 0 4'), Board('rnbqkbnr/ppppppp1/8/8/7P/5p1B/PPPPP3/RNBQK1

KeyboardInterrupt: 

In [1]:
# test_moves.py
import chess
import chess.pgn
import io
from features.extractor import FeatureExtractor
from analysis.stockfish_handler import StockfishHandler

# Moves input as a string (PGN format)
moves = """1. h4 h5 2. g4 hxg4 3. f3 gxf3 4. Bh3 fxe2 5. Qxe2 e5 6. Qd3 Qf6 7. Qc4 Rxh4 8. Qxc7 Bd6 9. Qxc8+ Ke7 10. Qxg8 Qf4 11. Qxg7 Rg4 12. Qh8 Qg3+ 13. Kf1 Rf4+ 14. Ke2 Rf2+ 15. Kd1 Rf1+ 16. Ke2 Qf2+ 17. Kd3 Qg3+ 18. Kc4 Rf4+ 19. Kd5 Rd4#)"""

# Create a file-like object from the moves string
pgn_io = io.StringIO(moves)

# Parse the game from the PGN string
game = chess.pgn.read_game(pgn_io)
if game is None:
    print("Failed to parse game!")
else:
    print("Game parsed successfully!\n")

# Print the PGN for verification
print("PGN:")
print(moves)
print("\n")

# Initialize the feature extractor and the Stockfish handler
feature_extractor = FeatureExtractor()
stockfish = StockfishHandler(path="stockfish", depth=20)

try:
    # Get positions from the game using the extractor method
    positions = feature_extractor._get_positions(game)
    # print("Positions:")
    # for i, board in enumerate(positions):
    #     print(f"Move {i}: {board.fen()}")
    # print("-" * 40)

    # Evaluate each position using Stockfish
    evals = [stockfish.evaluate_position(board, i) for i, board in enumerate(positions)]
    # print("\nEngine evaluations:")
    # for i, info in enumerate(evals):
    #     print(f"Ply {i}: {info.eval}")
    # print("-" * 40)

    # Extract features using both the game and engine evaluations
    features_with_engine = feature_extractor.extract_features(game, evals)
    print("\nFeatures with engine analysis:")
    for name, value in features_with_engine.__dict__.items():
        print(f"{name}: {value:.3f}")
finally:
    stockfish.close()


Game parsed successfully!

PGN:
1. h4 h5 2. g4 hxg4 3. f3 gxf3 4. Bh3 fxe2 5. Qxe2 e5 6. Qd3 Qf6 7. Qc4 Rxh4 8. Qxc7 Bd6 9. Qxc8+ Ke7 10. Qxg8 Qf4 11. Qxg7 Rg4 12. Qh8 Qg3+ 13. Kf1 Rf4+ 14. Ke2 Rf2+ 15. Kd1 Rf1+ 16. Ke2 Qf2+ 17. Kd3 Qg3+ 18. Kc4 Rf4+ 19. Kd5 Rd4#)


+0.3 -> -0.4 = Judgment.INACCURACY
-0.4 -> +0.5 = Judgment.INACCURACY
+0.5 -> -1.9 = Judgment.BLUNDER
-1.9 -> -1.9 = None
-1.9 -> -3.4 = Judgment.MISTAKE
-3.4 -> -2.1 = Judgment.INACCURACY
-2.1 -> -3.4 = Judgment.INACCURACY
-3.4 -> -3.4 = None
-3.4 -> -3.3 = None
-3.3 -> -2.3 = Judgment.INACCURACY
-2.3 -> -4.2 = Judgment.MISTAKE
-4.2 -> -3.2 = Judgment.INACCURACY
-3.2 -> -4.5 = Judgment.INACCURACY
-4.5 -> -3.3 = Judgment.INACCURACY
-3.3 -> -6.0 = Judgment.MISTAKE
-6.0 -> +3.9 = Judgment.BLUNDER
+3.9 -> +4.1 = None
+4.1 -> +4.9 = None
+4.9 -> +1.5 = Judgment.BLUNDER
+1.5 -> +4.1 = Judgment.BLUNDER
+4.1 -> +2.2 = Judgment.MISTAKE
+2.2 -> +8.2 = Judgment.BLUNDER
+8.2 -> -5.9 = Judgment.BLUNDER
-5.9 -> +4.0 = Judgment.BLUNDER
+